In [1]:
# Celda de verificación del entorno
import sys
import os
from datetime import datetime

print("🎉 ¡Bienvenido al Tutorial de Google ADK!")
print(f"\n📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"🐍 Versión de Python: {sys.version.split()[0]}")
print(f"💻 Entorno: Google Colab" if 'google.colab' in sys.modules else "💻 Entorno: Local")
print("\n✅ ¡Entorno listo para comenzar!")

🎉 ¡Bienvenido al Tutorial de Google ADK!

📅 Fecha: 2025-08-18 17:29
🐍 Versión de Python: 3.13.5
💻 Entorno: Local

✅ ¡Entorno listo para comenzar!


In [2]:
# Instalar Google ADK con soporte para LiteLLM
print("📦 Instalando Google ADK con LiteLLM...")
!pip install -q google-adk==1.4.2
!pip install -q litellm==1.73.0
!pip install -qU python-dotenv pydantic

print("\n✅ Instalación completada!")

# Verificar versiones
import sys
print(f"\n🐍 Python: {sys.version.split()[0]}")
!pip show google-adk litellm pydantic | grep -E "Name:|Version:"

📦 Instalando Google ADK con LiteLLM...

✅ Instalación completada!

🐍 Python: 3.13.5
Name: google-adk
Version: 1.4.2
Name: litellm
Version: 1.73.0
Name: pydantic
Version: 2.11.7


In [16]:
import os 
from getpass import getpass

if "GOOGLE_API_KEY" not in os.environ:
    google_key=getpass("google api key requerida:")
    if google_key:
        os.environ["GOOGLE_API_KEY"] = google_key
        os.environ["GOOGLE_GENAI_USE_VERTEXAI"]= "FALSE"


# Verificar configuración
print("\n📋 Estado de las API Keys:")
print(f"   Google: {'✅' if os.environ.get('GOOGLE_API_KEY') else '❌'}")


📋 Estado de las API Keys:
   Google: ✅


In [17]:
from dotenv import load_dotenv
# cargar variables de entorno desde env
load_dotenv(override=True)


True

In [18]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.genai import types



In [12]:
async def call_agent_async(query: str, runner, user_id, session_id):
    """Envía una consulta al agente e imprime la respuesta final."""
    print(f"\n>>> Consulta del usuario: {query}")

    # Prepara el mensaje del usuario en el formato de ADK
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "El agente no produjo una respuesta final." # Valor por defecto

    # Concepto clave: run_async ejecuta la lógica del agente y genera eventos.
    # Iteramos a través de los eventos para encontrar la respuesta final.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # Puedes descomentar la línea de abajo para ver *todos* los eventos durante la ejecución
        # print(f"  [Evento] Autor: {event.author}, Tipo: {type(event).__name__}, Final: {event.is_final_response()}, Contenido: {event.content}")

        # Concepto clave: is_final_response() marca el mensaje que concluye el turno.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Se asume que la respuesta de texto está en la primera parte
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Maneja posibles errores/escalamientos
                final_response_text = f"El agente escaló: {event.error_message or 'Sin mensaje específico.'}"
            # Agrega más validaciones aquí si es necesario (por ejemplo, códigos de error específicos)
            break # Deja de procesar eventos una vez encontrada la respuesta final

    print(f"<<< Respuesta del agente: {final_response_text}")

In [8]:
MODEL_GEMINI = "gemini-2.5-flash"

# Example: Defining the basic Agent
refranes_agent = LlmAgent(
    model=MODEL_GEMINI,
    name="refranes_agent",
    description="completa los refranes que el usuario empieza"
)

In [13]:
session_service = InMemorySessionService()
APP_NAME = "test_gemini"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
# Runner: This is the main component that manages the interaction with the agent.
runner_gemini = Runner(agent=refranes_agent,app_name=APP_NAME,session_service=session_service)

In [19]:
await call_agent_async("A caballo regalado...",
                           runner=runner_gemini,
                           user_id=USER_ID,
                           session_id=SESSION_ID)


>>> Consulta del usuario: A caballo regalado...
<<< Respuesta del agente: ...no se le mira el diente.
